In [13]:
import numpy as np
import pandas as pd

In [14]:
df = pd.read_csv("./clean_df.csv", usecols=["text", "sentiment"])
#df = pd.read_csv("./raw_df.csv", usecols=["text", "sentiment"])

In [15]:
train_valid_mask = ~df["sentiment"].isna()

In [16]:
from sklearn.model_selection import train_test_split

X = df["text"].values
y = df["sentiment"].values

X_train_valid = X[train_valid_mask]
y_train_valid = y[train_valid_mask]
X_test = X[~train_valid_mask]
y_test = y[~train_valid_mask]

X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, shuffle=True, random_state=42)

In [28]:
"""from sklearn.feature_extraction.text import TfidfVectorizer

max_features = 500000
#vectoriser = TfidfVectorizer(ngram_range=(1,2), stop_words="english", max_features=max_features)
vectoriser = TfidfVectorizer(ngram_range=(1,3), max_features=max_features)
vectoriser.fit(X_train)
X_train = vectoriser.transform(X_train)
X_valid  = vectoriser.transform(X_valid)"""

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

max_features = 500000
#vectoriser = TfidfVectorizer(ngram_range=(1,3), stop_words="english", max_features=max_features)
vectoriser = TfidfVectorizer(ngram_range=(1,3), max_features=max_features)
vectoriser.fit(X_train_valid)
X_train_valid = vectoriser.transform(X_train_valid)
X_test  = vectoriser.transform(X_test)

In [18]:
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

In [39]:
"""from sklearn.linear_model import LogisticRegression

LRmodel = LogisticRegression(C = 2, max_iter = 500, solver='liblinear', n_jobs=-1, random_state=42)
LRmodel.fit(X_train, y_train)
y_pred = LRmodel.predict(X_valid)

print(classification_report(y_valid, y_pred))
print(f1_score(y_valid, y_pred, average='macro'))"""

"from sklearn.linear_model import LogisticRegression\n\nLRmodel = LogisticRegression(C = 2, max_iter = 500, solver='liblinear', n_jobs=-1, random_state=42)\nLRmodel.fit(X_train, y_train)\ny_pred = LRmodel.predict(X_valid)\n\nprint(classification_report(y_valid, y_pred))\nprint(f1_score(y_valid, y_pred, average='macro'))"

In [40]:
"""from sklearn.svm import LinearSVC

SVCmodel = LinearSVC(random_state=42)
SVCmodel.fit(X_train, y_train)
y_pred = SVCmodel.predict(X_valid)

print(classification_report(y_valid, y_pred))
print(f1_score(y_valid, y_pred, average='macro'))"""

"from sklearn.svm import LinearSVC\n\nSVCmodel = LinearSVC(random_state=42)\nSVCmodel.fit(X_train, y_train)\ny_pred = SVCmodel.predict(X_valid)\n\nprint(classification_report(y_valid, y_pred))\nprint(f1_score(y_valid, y_pred, average='macro'))"

In [41]:
"""from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(100, random_state=42, criterion="entropy", max_features="log2", n_jobs=-1)
clf.fit(X_train, np.array(y_train))
y_pred = clf.predict(X_valid)

print(classification_report(y_valid, y_pred))
print(f1_score(y_valid, y_pred, average='macro'))"""

'from sklearn.ensemble import RandomForestClassifier\n\nclf = RandomForestClassifier(100, random_state=42, criterion="entropy", max_features="log2", n_jobs=-1)\nclf.fit(X_train, np.array(y_train))\ny_pred = clf.predict(X_valid)\n\nprint(classification_report(y_valid, y_pred))\nprint(f1_score(y_valid, y_pred, average=\'macro\'))'

In [19]:
from sklearn.model_selection import GridSearchCV

lr_param_grid = {
    "C" : [1, 2],
    "solver" : ["lbfgs", "liblinear"],
    "max_iter" : [100, 500, 1000, 5000],
    "random_state" : [42],
    "n_jobs" : [-1]
}

svc_param_grid = {
    "C" : [1, 2],
    "max_iter" : [100, 500, 1000, 5000],
    "random_state" : [42]
}

In [20]:
from sklearn.linear_model import LogisticRegression

lr_gs = GridSearchCV(LogisticRegression(), lr_param_grid, scoring="f1_macro", n_jobs=-1, cv=5)

lr_gs.fit(X_train_valid, y_train_valid)
lr_gs.best_score_

0.7740306040157504

In [21]:
y_pred_lr = lr_gs.predict(X_test)
pd.DataFrame(y_pred_lr).to_csv("output_lr.csv", index_label="Id", header=["Predicted"])
#pd.DataFrame(y_pred_lr).to_csv("output_lr_stopwords.csv", index_label="Id", header=["Predicted"])

In [22]:
from sklearn.svm import LinearSVC

svc_gs = GridSearchCV(LinearSVC(), svc_param_grid, scoring="f1_macro", n_jobs=-1, cv=5)

svc_gs.fit(X_train_valid, y_train_valid)
svc_gs.best_score_

0.768949545810272

In [23]:
y_pred_svc = svc_gs.predict(X_test)
pd.DataFrame(y_pred_svc).to_csv("output_svc.csv", index_label="Id", header=["Predicted"])

In [47]:
lr_gs.best_estimator_

LogisticRegression(C=2, n_jobs=-1, random_state=42, solver='liblinear')

In [48]:
svc_gs.best_estimator_

LinearSVC(C=1, max_iter=100, random_state=42)

In [24]:
lr_df = pd.concat([pd.DataFrame(lr_gs.cv_results_["params"]),pd.DataFrame(lr_gs.cv_results_["mean_test_score"], columns=["f1_macro"])],axis=1)

In [50]:
lr_df

,C,max_iter,n_jobs,random_state,solver,f1_macro
0,1,100,-1,42,lbfgs,0.795671
1,1,100,-1,42,liblinear,0.795733
2,1,500,-1,42,lbfgs,0.795751
3,1,500,-1,42,liblinear,0.795733
4,1,1000,-1,42,lbfgs,0.795751
5,1,1000,-1,42,liblinear,0.795733
6,1,5000,-1,42,lbfgs,0.795751
7,1,5000,-1,42,liblinear,0.795733
8,2,100,-1,42,lbfgs,0.800260
9,2,100,-1,42,liblinear,0.800507


In [25]:
svc_df = pd.concat([pd.DataFrame(svc_gs.cv_results_["params"]),pd.DataFrame(svc_gs.cv_results_["mean_test_score"], columns=["f1_macro"])],axis=1)

In [52]:
svc_df

,C,max_iter,random_state,f1_macro
0,1,100,42,0.795516
1,1,500,42,0.795516
2,1,1000,42,0.795516
3,1,5000,42,0.795516
4,2,100,42,0.788191
5,2,500,42,0.788191
6,2,1000,42,0.788191
7,2,5000,42,0.788191


In [26]:
pd.DataFrame(svc_df).to_csv("grid_svc_stopwords.csv", index_label="Id", header=["C", "max_iter", "random_state", "f1_macro"])

In [27]:
pd.DataFrame(lr_df).to_csv("grid_lr_stopwords.csv", index_label="Id", header=["C", "max_iter", "n_jobs", "random_state", "solver", "f1_macro"])

In [55]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

In [30]:
f1_score_dict = dict()

for max_features in [100, 500, 1000, 5000, 10000, 50000, 100000, 500000]:
    X = df["text"].values
    y = df["sentiment"].values

    X_train_valid = X[train_valid_mask]
    y_train_valid = y[train_valid_mask]
    X_test = X[~train_valid_mask]
    y_test = y[~train_valid_mask]
    
    X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, shuffle=True, random_state=42)

    vectoriser = TfidfVectorizer(ngram_range=(1,3), max_features=max_features)
    #vectoriser = TfidfVectorizer(ngram_range=(1,3), max_features=max_features, stop_words="english")
    vectoriser.fit(X_train)
    X_train = vectoriser.transform(X_train)
    X_valid  = vectoriser.transform(X_valid)
    
    SVCmodel = LinearSVC(C=1, max_iter=100, random_state=42)
    SVCmodel.fit(X_train, y_train)
    y_pred = SVCmodel.predict(X_valid)
    f1_svc = f1_score(y_valid, y_pred, average='macro')
    
    LRmodel = LogisticRegression(C=2, n_jobs=-1, random_state=42)
    LRmodel.fit(X_train, y_train)
    y_pred = LRmodel.predict(X_valid)
    f1_lr = f1_score(y_valid, y_pred, average='macro')
    
    f1_score_dict[max_features] = [f1_lr, f1_svc]    

In [31]:
pd.DataFrame.from_dict(f1_score_dict, orient='index', columns=["LogisticRegressor", "LinearSVC"]).to_csv("f1_res.csv")
#pd.DataFrame.from_dict(f1_score_dict, orient='index', columns=["LogisticRegressor", "LinearSVC"]).to_csv("f1_res_raw.csv")
#pd.DataFrame.from_dict(f1_score_dict, orient='index', columns=["LogisticRegressor", "LinearSVC"]).to_csv("f1_res_stopwords.csv")

In [58]:
ngram_compare_list = list()
max_features = 500000
ngram_list = [(1,1), (1,2), (1,3)]

for ngram in ngram_list:
    f1_score_dict = dict()

    for max_features in [100, 500, 1000, 5000, 10000, 50000, 100000, 500000]:
        X = df["text"].values
        y = df["sentiment"].values

        X_train_valid = X[train_valid_mask]
        y_train_valid = y[train_valid_mask]
        X_test = X[~train_valid_mask]
        y_test = y[~train_valid_mask]

        X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, shuffle=True, random_state=42)

        vectoriser = TfidfVectorizer(ngram_range=ngram, max_features=max_features)
        vectoriser.fit(X_train)
        X_train = vectoriser.transform(X_train)
        X_valid  = vectoriser.transform(X_valid)

        SVCmodel = LinearSVC(C=1, max_iter=100, random_state=42)
        SVCmodel.fit(X_train, y_train)
        y_pred = SVCmodel.predict(X_valid)
        f1_svc = f1_score(y_valid, y_pred, average='macro')

        LRmodel = LogisticRegression(C=2, n_jobs=-1, random_state=42)
        LRmodel.fit(X_train, y_train)
        y_pred = LRmodel.predict(X_valid)
        f1_lr = f1_score(y_valid, y_pred, average='macro')

        f1_score_dict[max_features] = [f1_lr, f1_svc]

    ngram_compare_list.append(f1_score_dict)

In [59]:
i = 0
for elem in ngram_compare_list:
    pd.DataFrame.from_dict(ngram_compare_list[i], orient='index', columns=["LogisticRegressor", "LinearSVC"]).to_csv(f"ngram_compare_{i}.csv")
    i = i + 1

In [10]:
from sklearn.neighbors import KNeighborsClassifier

knc_param_grid = {
    "n_neighbors" : [5, 10],
    "weights" : ["uniform", "distance"]
}

knc_gs = GridSearchCV(KNeighborsClassifier(), knc_param_grid, scoring="f1_macro", n_jobs=4, cv=5, verbose=5)

knc_gs.fit(X_train_valid, y_train_valid)
knc_gs.best_score_

Fitting 5 folds for each of 4 candidates, totalling 20 fits


0.5377209932230912

In [11]:
knc_df = pd.concat([pd.DataFrame(knc_gs.cv_results_["params"]),pd.DataFrame(knc_gs.cv_results_["mean_test_score"], columns=["f1_macro"])],axis=1)

In [12]:
pd.DataFrame(knc_df).to_csv("grid_knc.csv", index_label="Id", header=["n_neighbors", "weights", "f1_macro"])